In [1]:
import pandas as pd 
import numpy as np
anime=pd.read_csv('anime.csv')
anime.genre.replace(np.nan,'1',inplace=True)
anime.genre.isnull().sum()
anime.head(4)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572


In [2]:
import random

anime_rd=random.choices(anime['anime_id'],k=1000)
anime=anime[anime['anime_id'].isin(anime_rd)]
print(anime.shape)

(964, 7)


In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

anime_genre=anime.join(pd.DataFrame(mlb.fit_transform(anime['genre'].str.split(',')),columns=mlb.classes_,index=anime.index))
del anime_genre['1']
anime_genre.head(2)

,anime_id,name,genre,type,episodes,rating,members,Adventure,Cars,Comedy,...,Mystery,Psychological,Romance,School,Sci-Fi,Seinen,Shounen,Slice of Life,Sports,Supernatural
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
anime_dummy=pd.get_dummies(anime_genre,columns=['type','type']).set_index('anime_id')
anime_dummy.drop(columns=['members','episodes','rating','name','genre'],inplace=True)
anime_dummy.head(3)

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,type_ONA,type_OVA,type_Special,type_TV,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
anime_id,,,,,,,,,,,,,,,,,,,,,
9253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
32935,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
820,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


<h2>TF-IDF</h2>
<blockquote>Giving weight to each attributes by computing tf-idf

In [5]:
#TF
anime_tf=anime_dummy.apply(lambda x: x/anime_dummy.sum(axis=1))
anime_tf.head(3)

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,type_ONA,type_OVA,type_Special,type_TV,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
anime_id,,,,,,,,,,,,,,,,,,,,,
9253,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,0.000000,0.0,0.250000
32935,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.142857,0.0,0.0,0.0,0.000000,0.0,0.142857
820,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.000000


In [6]:
#IDF
freq=anime_dummy.sum(axis=0)
idf=(len(anime_dummy)/freq).apply(np.log2)

#weight
weight_anime=anime_tf.mul(idf.values)

<h2>User-item matrix</h2>
<blockquote>By pivoting rating table.

In [7]:
rating=pd.read_csv('rating.csv')
print('mean ',rating.rating.mean())
print('median ',rating.rating.quantile(0.5))
print('lower boundary ',rating.rating.min())
print('upper boundary',rating.rating.max())
rating['user_id']=rating['user_id'].astype(str)
rating.head(5)

mean  6.144029546937656
median  7.0
lower boundary  -1
upper boundary 10


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [8]:
rating=rating[rating['anime_id'].isin(anime_rd)]

In [9]:
rating_df=pd.pivot_table(rating,values='rating',columns='user_id',index='anime_id')
rating_df

user_id,1,10,1000,10000,10001,10002,10003,10004,10005,10006,...,9988,9989,999,9990,9992,9993,9994,9995,9996,9998
anime_id,,,,,,,,,,,,,,,,,,,,,
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h3>Computing User preference</h3>

In [10]:
#user preference
user_id=input('userid : ')
user_metric=pd.DataFrame(columns=anime_tf.columns)

temp=anime_tf.mul(rating_df.loc[:,user_id],axis=0)
user_metric.loc[user_id]=temp.sum(axis=0)
user_metric

userid : 1


,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,type_ONA,type_OVA,type_Special,type_TV,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
1,0.0,0.0,-0.577381,0.0,1.25,-0.267857,0.532143,-0.267857,0.0,0.446429,...,0.0,-0.267857,-0.366667,-0.346429,0.0,0.0,0.0,-0.267857,-0.366667,-0.346429


In [17]:
#predict
pred_df=pd.DataFrame()
temp2=weight_anime.mul(user_metric.loc[:user_id],axis=1)
pred_df[user_id]=temp2.sum(axis=1)
pred_df

SyntaxError: invalid syntax (<ipython-input-17-107132248c95>, line 5)

In [26]:
def RecommendAnime(user):
    user=str(user)
    user_metric=pd.DataFrame(columns=anime_tf.columns)

    temp=anime_tf.mul(rating_df.loc[:,user],axis=0)
    user_metric.loc[user]=temp.sum(axis=0)
    
    pred_df=pd.DataFrame()
    temp2=weight_anime.mul(user_metric.loc[:user],axis=1)
    pred_df[user]=temp2.sum(axis=1)
    return pred_df

In [25]:
RecommendAnime('1')

,1
17,0.0
21,0.0
27,0.0
60,0.0
74,0.0
...,...
34284,0.0
34350,0.0
34393,0.0
34451,0.0
